# Feature Tracking - Setup Tables and Views

**Purpose**: One-time setup for feature tracking tables and views

**What this creates**:
- ✅ `feature_releases_roadmap` - Feature releases from Fabric GPS API (with roadmap)
- ✅ `preview_features_active` - Detected activated preview features
- ✅ `feature_alerts` - Alerts for new/risky preview features
- ✅ Helper SQL views for easy querying

In [ ]:
from datetime import datetime
import pandas as pd

## Step 1: Create `feature_releases_roadmap` Table

In [ ]:
print("🔄 Creating table: feature_releases_roadmap")
print("=" * 70)

spark.sql("""
    CREATE TABLE IF NOT EXISTS feature_releases_roadmap (
        feature_id STRING NOT NULL,
        feature_name STRING NOT NULL,
        feature_description STRING,
        workload STRING,
        product_name STRING,
        release_date TIMESTAMP,
        release_type STRING,
        release_status STRING,
        is_preview BOOLEAN NOT NULL,
        is_planned BOOLEAN NOT NULL,
        is_shipped BOOLEAN NOT NULL,
        last_modified TIMESTAMP NOT NULL,
        source_url STRING,
        source STRING,
        extracted_date TIMESTAMP NOT NULL
    )
    USING DELTA
""")

print("✅ Table created: feature_releases_roadmap")
print("   Schema: 15 columns")
print("   💡 Includes planned/future features and historical tracking")

## Step 2: Create `preview_features_active` Table

In [ ]:
print("\n🔄 Creating table: preview_features_active")
print("=" * 70)

spark.sql("""
    CREATE TABLE IF NOT EXISTS preview_features_active (
        setting_name STRING NOT NULL,
        feature_id STRING NOT NULL,
        feature_name STRING NOT NULL,
        workload STRING,
        similarity_score DOUBLE NOT NULL,
        is_enabled BOOLEAN NOT NULL,
        delegate_to_tenant BOOLEAN,
        detected_date TIMESTAMP NOT NULL,
        release_date TIMESTAMP,
        release_status STRING,
        source_url STRING,
        days_since_release INT
    )
    USING DELTA
""")

print("✅ Table created: preview_features_active")
print("   Schema: 12 columns")

## Step 3: Create `feature_alerts` Table

In [ ]:
print("\n🔄 Creating table: feature_alerts")
print("=" * 70)

spark.sql("""
    CREATE TABLE IF NOT EXISTS feature_alerts (
        alert_id STRING NOT NULL,
        feature_id STRING NOT NULL,
        feature_name STRING NOT NULL,
        workload STRING,
        alert_type STRING NOT NULL,
        severity STRING NOT NULL,
        message STRING NOT NULL,
        setting_name STRING,
        similarity_score DOUBLE,
        days_since_release INT,
        alert_date TIMESTAMP NOT NULL,
        acknowledged BOOLEAN NOT NULL,
        acknowledged_date TIMESTAMP,
        acknowledged_by STRING
    )
    USING DELTA
""")

print("✅ Table created: feature_alerts")
print("   Schema: 14 columns")

## Step 4: Create Helper SQL Views

In [ ]:
print("\n🔄 Creating helper SQL views...")
print("=" * 70)

# View 1: Roadmap Upcoming Features
spark.sql("""
    CREATE OR REPLACE VIEW vw_roadmap_upcoming AS
    SELECT 
        feature_name,
        feature_description,
        product_name,
        workload,
        release_type,
        release_status,
        release_date,
        is_preview,
        is_planned,
        last_modified,
        CASE 
            WHEN release_date IS NULL THEN NULL
            ELSE DATEDIFF(release_date, CURRENT_DATE())
        END as days_until_release
    FROM feature_releases_roadmap
    WHERE is_planned = true
      AND (release_date IS NULL OR release_date >= CURRENT_DATE())
    ORDER BY release_date ASC NULLS LAST, last_modified DESC
""")
print("✅ vw_roadmap_upcoming - Planned/upcoming features")

# View 2: Active Preview Features
spark.sql("""
    CREATE OR REPLACE VIEW vw_active_preview_features AS
    SELECT 
        feature_name,
        workload,
        setting_name,
        days_since_release,
        similarity_score,
        release_date,
        detected_date,
        is_enabled
    FROM preview_features_active
    WHERE is_enabled = true
    ORDER BY detected_date DESC
""")
print("✅ vw_active_preview_features - Currently enabled previews")

# View 3: Critical Alerts
spark.sql("""
    CREATE OR REPLACE VIEW vw_critical_alerts AS
    SELECT 
        alert_id,
        feature_name,
        workload,
        alert_type,
        severity,
        message,
        alert_date,
        acknowledged
    FROM feature_alerts
    WHERE acknowledged = false 
      AND severity IN ('Critical', 'Warning')
    ORDER BY 
        CASE severity 
            WHEN 'Critical' THEN 1 
            WHEN 'Warning' THEN 2 
            ELSE 3 
        END,
        alert_date DESC
""")
print("✅ vw_critical_alerts - Unacknowledged critical/warning alerts")

# View 4: Feature Release Timeline
spark.sql("""
    CREATE OR REPLACE VIEW vw_feature_timeline AS
    SELECT 
        feature_name,
        product_name,
        workload,
        release_type,
        release_status,
        is_preview,
        is_planned,
        is_shipped,
        release_date,
        CASE 
            WHEN release_date IS NULL THEN NULL
            ELSE DATEDIFF(CURRENT_DATE(), release_date)
        END as days_since_release,
        last_modified
    FROM feature_releases_roadmap
    ORDER BY release_date DESC NULLS LAST
""")
print("✅ vw_feature_timeline - Complete release timeline")

print("\n✅ All SQL views created successfully")

## ✅ Setup Complete!

In [ ]:
print("\n" + "=" * 70)
print("🎉 FEATURE TRACKING SETUP COMPLETED!")
print("=" * 70)

# Verify tables
tables = ["feature_releases_roadmap", "preview_features_active", "feature_alerts"]
print("\n📋 Tables created:")
for table in tables:
    try:
        count = spark.read.format("delta").table(table).count()
        print(f"  ✅ {table}: {count} rows")
    except Exception as e:
        print(f"  ❌ {table}: ERROR - {e}")

# Verify views
views = ["vw_roadmap_upcoming", "vw_active_preview_features", "vw_critical_alerts", "vw_feature_timeline"]
print("\n📋 Views created:")
for view in views:
    try:
        spark.sql(f"SELECT * FROM {view} LIMIT 1")
        print(f"  ✅ {view}")
    except Exception as e:
        print(f"  ❌ {view}: ERROR - {e}")

print("\n" + "=" * 70)
print("📚 Next Step:")
print("=" * 70)
print("\n  → Run 'Load_Feature_Tracking' notebook to populate the tables")
print("\n💡 Schedule Load_Feature_Tracking to run daily for continuous monitoring")
print("\n" + "=" * 70)